In [10]:
import gym
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch 
from utils import plot_learning_curve

from torchvision import datasets, transforms, models
import torch 
from torch import nn, optim, utils, device as device_, cuda
import numpy as np

### Linear Deep Q network class

In [11]:
class LinearDQNetwork(nn.Module): #linear deep q netwrok
    def __init__(self, lr, n_actions, input_dims):
        super(LinearDQNetwork, self).__init__()
        
        self.fc1 = nn.Linear(*input_dims, 128) #the fist fully connected layer takes in the inputs and retirns 128 layers
        self.fc2 = nn.Linear(128, n_actions) #the second fully connected layer takes in the 128 layers as 
        #inputs and returns value of actions (estimate for Q)
        
        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss = nn.MSELoss()
        self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cup') #query torch device if gpu not avaliable results in cpu
        self.to(self.device) # send all network to device
        
        
    def forwardpass(self, state): #forward pass the states and return actions
        layer1 = F.relu(self.fc1(state))
        actions = self.fc2(layer1)
        
        return actions
    
    

### Agent

In [14]:
class Agent():
    def __init__(self, input_dims, n_actions, lr, gamma=0.99,
                epsilon=1.0, eps_dec=1e-5, eps_min=0.01):

        self.lr = lr
        self.input_dims = input_dims
        self.n_actions = n_actions
        self.gamma = gamma
        self.epsilon = epsilon
        self.eps_dec = eps_dec
        self.eps_min = eps_min
        self.action_space = [i for i in range(self.n_actions)]

        self.Q = LinearDQNetwork(self.lr, self.n_actions, self.input_dims)
        
        
#         def choose_action(self, observation):
#             if np.rnadom.random() > self.epsilon: #random value for greedy action, if greater than epsilion take a random action
                
#                 state = torch.tensor(observation, dtype=torch.float).to(self.Q.device) #obsertaion to torch tensor and send to device
#                 actions = self.Q.forwardpass(state) #value of actions for a given state
#                 action = torch.argmax(actions).item() 
#             else:
#                 actions = np.random.choice(self.action_space)
#                 return action
        
        def choose_action(self, observation):
            if np.random.random() > self.epsilon:
                state  = T.tensor(observation, dtype=T.float).to(self.Q.device)
                actions = self.Q.forward(state)
                action = T.argmax(actions).item()
            else:
                action = np.random.choice(self.action_space)

            
#             #find the max of each action - derefrence by .iteam()
#             #since torch.argmax(actions) retusn torhc tensors and we need numpy arrays which is required for open ai gym
#         def decreament_epsilon(self):
#             self.epsilon = self.epsilon - self.eps_dec if self.epsilon > self.eps_min else self.eps_min
    
#     # decrase epsilion linarly - it will reach minimum (finite) over time
    
#       #how the agent will learn from its experince - current action state reward and new state as inputs

#         def learn(self, state, actions, reward, state_new):
#             self.Q.optimizer.zero_grad() #zero gradients
#             state = torch.tensors(state, dtype=torch.float).to(self.Q.device) #arrays to cuda tensors
#             actions = torch.tensors(action).to(self.Q.device)
#             rewards = torch.tensor(reward).to(self.Q.device)
#             states_new = torch.tensor(state_new, dtype=torch.float).to(self.Q.device) #all numpy arrays as torch tensors


#             #feed forward to update Q estimate
#             Q_estimate = self.Q.forward(states)[actions] #predicted values for currnt state of the environment 
#             #the change between the action taken and total possible actions
            
#             #the maximum actions of the resulting state
#             Q_estimate_next = self.Q.forward(state_next).max() 
            
#             #direction to move 
#             Q_target = reward + self.gamma*Q_estimate_next
            
#             #distance between the action taken and the maximum action the agent could take (mean squred error)
#             loss = self.Q.loss(Q_target, Q_estimate).to(self.Q.device)
            
#             #backpropagte 
#             loss.backward.setp()
#             #setp optimizer
#             self.Q.optimizer.step()
#             self.decreament_epsilon()

    def decrement_epsilon(self):
        self.epsilon = self.epsilon - self.eps_dec \
                if self.epsilon > self.eps_min else self.eps_min

    def learn(self, state, action, reward, state_):
        self.Q.optimizer.zero_grad()
        states = T.tensor(state, dtype=T.float).to(self.Q.device)
        actions = T.tensor(action).to(self.Q.device)
        rewards = T.tensor(reward).to(self.Q.device)
        states_ = T.tensor(state_, dtype=T.float).to(self.Q.device)

        q_pred = self.Q.forward(states)[actions]

        q_next = self.Q.forward(states_).max()

        q_target = reward + self.gamma*q_next

        loss = self.Q.loss(q_target, q_pred).to(self.Q.device)
        loss.backward()
        self.Q.optimizer.step()
        self.decrement_epsilon()
        
    
            

## Training loop and test

In [ ]:
if __name__ == "__main__":
    env = gym.make("CartPole-v1")
    n_games = 10000
    scores = []
    eps_history = []

    agent = Agent(input_dims=env.observation_space.shape, 
                n_actions=env.action_space.n, lr=0.0001)

    for i in range(n_games):
        score = 0
        done = False
        obs = env.reset()

        while not done:
            action = agent.choose_action(obs)
            obs_, reward, done, info = env.step(action)
            score += reward
            agent.learn(obs, action, reward, obs_)
            obs = obs_
        scores.append(score)
        eps_history.append(agent.epsilon)

        if i % 100 == 0:
            avg_score = np.mean(scores[-100:])
            print("episode", i, "score %.1f average score %.1f epsilon %.2f" %
               (score, avg_score, agent.epsilon))

episode 0 score 17.0 average score 17.0 epsilon 1.00
episode 100 score 42.0 average score 21.5 epsilon 0.98
episode 200 score 35.0 average score 21.9 epsilon 0.96
episode 300 score 52.0 average score 20.2 epsilon 0.94
episode 400 score 12.0 average score 24.3 epsilon 0.91
episode 500 score 12.0 average score 21.8 epsilon 0.89
episode 600 score 24.0 average score 22.3 epsilon 0.87
episode 700 score 15.0 average score 22.6 epsilon 0.85


In [17]:
import torch as T

class LinearDeepQNetwork(nn.Module):
    def __init__(self, lr, n_actions, input_dims):
        super(LinearDeepQNetwork, self).__init__()

        self.fc1 = nn.Linear(*input_dims, 128)
        self.fc2 = nn.Linear(128, n_actions)

        self.optimizer = optim.Adam(self.parameters(), lr=lr)
        self.loss  = nn.MSELoss()
        self.device = T.device('cuda:0' if T.cuda.is_available() else 'cpu')
        self.to(self.device)

    def forward(self, state):
        layer1 = F.relu(self.fc1(state))
        action = self.fc2(layer1)

        return action

class Agent():
    def __init__(self, input_dims, n_actions, lr, gamma=0.99,
                epsilon=1.0, eps_dec=1e-5, eps_min=0.01):

        self.lr = lr
        self.input_dims = input_dims
        self.n_actions = n_actions
        self.gamma = gamma
        self.epsilon = epsilon
        self.eps_dec = eps_dec
        self.eps_min = eps_min
        self.action_space = [i for i in range(self.n_actions)]

        self.Q = LinearDeepQNetwork(self.lr, self.n_actions, self.input_dims)

    def choose_action(self, observation):
        if np.random.random() > self.epsilon:
            state  = T.tensor(observation, dtype=T.float).to(self.Q.device)
            actions = self.Q.forward(state)
            action = T.argmax(actions).item()
        else:
            action = np.random.choice(self.action_space)

        return action

    def decrement_epsilon(self):
        self.epsilon = self.epsilon - self.eps_dec \
                if self.epsilon > self.eps_min else self.eps_min

    def learn(self, state, action, reward, state_):
        self.Q.optimizer.zero_grad()
        states = T.tensor(state, dtype=T.float).to(self.Q.device)
        actions = T.tensor(action).to(self.Q.device)
        rewards = T.tensor(reward).to(self.Q.device)
        states_ = T.tensor(state_, dtype=T.float).to(self.Q.device)

        q_pred = self.Q.forward(states)[actions]

        q_next = self.Q.forward(states_).max()

        q_target = reward + self.gamma*q_next

        loss = self.Q.loss(q_target, q_pred).to(self.Q.device)
        loss.backward()
        self.Q.optimizer.step()
        self.decrement_epsilon()

if __name__ == "__main__":
    env = gym.make("CartPole-v1")
    n_games = 10000
    scores = []
    eps_history = []

    agent = Agent(input_dims=env.observation_space.shape, 
                n_actions=env.action_space.n, lr=0.0001)

    for i in range(n_games):
        score = 0
        done = False
        obs = env.reset()

        while not done:
            action = agent.choose_action(obs)
            obs_, reward, done, info = env.step(action)
            score += reward
            agent.learn(obs, action, reward, obs_)
            obs = obs_
        scores.append(score)
        eps_history.append(agent.epsilon)

        if i % 100 == 0:
            avg_score = np.mean(scores[-100:])
            print("episode", i, "score %.1f average score %.1f epsilon %.2f" %
               (score, avg_score, agent.epsilon))

episode 0 score 31.0 average score 31.0 epsilon 1.00
episode 100 score 22.0 average score 21.7 epsilon 0.98
episode 200 score 28.0 average score 19.7 epsilon 0.96
episode 300 score 14.0 average score 21.3 epsilon 0.94
episode 400 score 18.0 average score 21.2 epsilon 0.92
episode 500 score 11.0 average score 23.9 epsilon 0.89
episode 600 score 22.0 average score 19.7 epsilon 0.87
episode 700 score 14.0 average score 22.6 epsilon 0.85
episode 800 score 21.0 average score 22.0 epsilon 0.83
episode 900 score 26.0 average score 23.7 epsilon 0.80
episode 1000 score 20.0 average score 26.1 epsilon 0.78
episode 1100 score 22.0 average score 24.5 epsilon 0.75
episode 1200 score 10.0 average score 24.2 epsilon 0.73
episode 1300 score 11.0 average score 23.6 epsilon 0.71
episode 1400 score 18.0 average score 28.0 epsilon 0.68
episode 1500 score 44.0 average score 29.0 epsilon 0.65
episode 1600 score 90.0 average score 23.8 epsilon 0.62
episode 1700 score 15.0 average score 27.1 epsilon 0.60
epis